This is my reboot of markham's course. 

In [4]:
from sklearn.datasets import load_iris
iris = load_iris()

In [5]:
X = iris.data
y = iris.target

In [6]:
print(X.shape)
print(y.shape)

(150, 4)
(150,)


In [7]:
import pandas as pd
pd.DataFrame(X, columns=iris.feature_names).head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [8]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

knn.fit(X, y)

/Users/michaelreinhardme.com/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [9]:
knn.predict([[3,5,4,2]])

array([1])

In [10]:
simple_train = ['call you tonight', 'Call me a cab', 'please call me....PLEASE!']

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [12]:
vect.fit(simple_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [13]:
vect.get_feature_names()

['cab', 'call', 'me', 'please', 'tonight', 'you']

In [15]:
simple_train_dtm = vect.transform(simple_train)
simple_train_dtm

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [16]:
simple_train_dtm.toarray()

array([[0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0],
       [0, 1, 1, 2, 0, 0]])

In [18]:
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,0,0,1,1
1,1,1,1,0,0,0
2,0,1,1,2,0,0


In [19]:
print(simple_train_dtm)

  (0, 1)	1
  (0, 4)	1
  (0, 5)	1
  (1, 0)	1
  (1, 1)	1
  (1, 2)	1
  (2, 1)	1
  (2, 2)	1
  (2, 3)	2


In [20]:
simple_test = ["please don't call me"]

In [23]:
simple_test_dtm = vect.transform(simple_test)
simple_test_dtm.toarray()

array([[0, 1, 1, 1, 0, 0]])

In [25]:
pd.DataFrame(simple_test_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,1,1,0,0


In [28]:
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/sms.tsv'

In [29]:
sms = pd.read_table(url, header=None, names=['label','message'])

In [30]:
sms.shape

(5572, 2)

In [31]:
sms.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [32]:
sms.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [33]:
sms['label_num'] = sms.label.map({'ham':0, 'spam':1})

In [34]:
sms.head()

,label,message,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [35]:
X = sms.message
y = sms.label_num
print(X.shape)
print(y.shape)

(5572,)
(5572,)


In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [40]:
print(X_train.shape)
print(X_test.shape)

(4179,)
(1393,)


In [44]:
vect = CountVectorizer()

In [45]:
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

In [46]:
X_train_dtm = vect.fit_transform(X_train)

In [47]:
X_train_dtm

<4179x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 55209 stored elements in Compressed Sparse Row format>

In [48]:
X_test_dtm = vect.transform(X_test)
X_test_dtm

<1393x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 17604 stored elements in Compressed Sparse Row format>

Part 5: Building a Naive Bayes model

In [52]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [53]:
%time nb.fit(X_train_dtm, y_train)

CPU times: user 5.33 ms, sys: 3.69 ms, total: 9.02 ms
Wall time: 10.6 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [54]:
y_pred_class = nb.predict(X_test_dtm)

In [55]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.9885139985642498

In [56]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[1203,    5],
       [  11,  174]])

In [57]:
X_test[y_test < y_pred_class]

574               Waiting for your call.
3375             Also andros ice etc etc
45      No calls..messages..missed calls
3415             No pic. Please re-send.
1988    No calls..messages..missed calls
Name: message, dtype: object

In [58]:
X_test[y_test > y_pred_class]

3132    LookAtMe!: Thanks for your purchase of a video...
5       FreeMsg Hey there darling it's been 3 week's n...
3530    Xmas & New Years Eve tickets are now on sale f...
684     Hi I'm sue. I am 20 years old and work as a la...
1875    Would you like to see my XXX pics they are so ...
1893    CALL 09090900040 & LISTEN TO EXTREME DIRTY LIV...
4298    thesmszone.com lets you send free anonymous an...
4949    Hi this is Amy, we will be sending you a free ...
2821    INTERFLORA - It's not too late to order Inter...
2247    Hi ya babe x u 4goten bout me?' scammers getti...
4514    Money i have won wining number 946 wot do i do...
Name: message, dtype: object

In [59]:
X_test[3132]

"LookAtMe!: Thanks for your purchase of a video clip from LookAtMe!, you've been charged 35p. Think you can do better? Why not send a video in a MMSto 32323."

In [60]:
y_pred_prob = nb.predict_proba(X_test_dtm)[:,1]
y_pred_prob

array([2.87744864e-03, 1.83488846e-05, 2.07301295e-03, ...,
       1.09026171e-06, 1.00000000e+00, 3.98279868e-09])

In [61]:
metrics.roc_auc_score(y_test, y_pred_prob)

0.9866431000536962

In [62]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

/Users/michaelreinhardme.com/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [63]:
%time logreg.fit(X_train_dtm, y_train)

CPU times: user 35.1 ms, sys: 5.73 ms, total: 40.9 ms
Wall time: 49.1 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [64]:
y_pred_class = logreg.predict(X_test_dtm)

In [65]:
y_pred_proba = logreg.predict_proba(X_test_dtm)[:,1]
y_pred_proba

array([0.01269556, 0.00347183, 0.00616517, ..., 0.03354907, 0.99725053,
       0.00157706])

In [66]:
metrics.accuracy_score(y_test, y_pred_class)

0.9877961234745154

In [67]:
metrics.roc_auc_score(y_test, y_pred_proba)

0.9936817612314301

Part 7: Calculating Spamminess of each token

In [68]:
X_train_tokens = vect.get_feature_names()
len(X_train_tokens)

7456

In [69]:
print(X_train_tokens[0:50])

['00', '000', '008704050406', '0121', '01223585236', '01223585334', '0125698789', '02', '0207', '02072069400', '02073162414', '02085076972', '021', '03', '04', '0430', '05', '050703', '0578', '06', '07', '07008009200', '07090201529', '07090298926', '07123456789', '07732584351', '07734396839', '07742676969', '0776xxxxxxx', '07781482378', '07786200117', '078', '07801543489', '07808', '07808247860', '07808726822', '07815296484', '07821230901', '07880867867', '0789xxxxxxx', '07946746291', '0796xxxxxx', '07973788240', '07xxxxxxxxx', '08', '0800', '08000407165', '08000776320', '08000839402', '08000930705']


In [70]:
print(X_train_tokens[-50:])

['yer', 'yes', 'yest', 'yesterday', 'yet', 'yetunde', 'yijue', 'ym', 'ymca', 'yo', 'yoga', 'yogasana', 'yor', 'yorge', 'you', 'youdoing', 'youi', 'youphone', 'your', 'youre', 'yourjob', 'yours', 'yourself', 'youwanna', 'yowifes', 'yoyyooo', 'yr', 'yrs', 'ything', 'yummmm', 'yummy', 'yun', 'yunny', 'yuo', 'yuou', 'yup', 'zac', 'zaher', 'zealand', 'zebra', 'zed', 'zeros', 'zhong', 'zindgi', 'zoe', 'zoom', 'zouk', 'zyada', 'èn', '〨ud']


In [72]:
nb.feature_count_

array([[ 0.,  0.,  0., ...,  1.,  1.,  1.],
       [ 5., 23.,  2., ...,  0.,  0.,  0.]])

In [73]:
nb.feature_count_.shape

(2, 7456)

In [74]:
ham_token_count = nb.feature_count_[0, :]
ham_token_count

array([0., 0., 0., ..., 1., 1., 1.])

In [75]:
spam_token_count = nb.feature_count_[1, :]
spam_token_count

array([ 5., 23.,  2., ...,  0.,  0.,  0.])

In [76]:
tokens = pd.DataFrame({'token':X_train_tokens, 'ham':ham_token_count, 'spam':spam_token_count})

In [77]:
tokens.sample(5, random_state=6)

,ham,spam,token
6977,64.0,2.0,very
4521,1.0,1.0,nasty
6999,0.0,1.0,villa
1266,1.0,0.0,beloved
6522,0.0,2.0,textoperator


In [79]:
tokens['ham'] = tokens.ham + 1
tokens['spam'] = tokens.spam + 1
tokens.sample(5, random_state=6)

,ham,spam,token
6977,66.0,4.0,very
4521,3.0,3.0,nasty
6999,2.0,3.0,villa
1266,3.0,2.0,beloved
6522,2.0,4.0,textoperator


In [80]:
nb.class_count_

array([3617.,  562.])

In [81]:
tokens['ham'] = tokens.ham / nb.class_count_[0]
tokens['spam'] = tokens.spam / nb.class_count_[1]
tokens.sample(5, random_state=6)

,ham,spam,token
6977,0.018247,0.007117,very
4521,0.000829,0.005338,nasty
6999,0.000553,0.005338,villa
1266,0.000829,0.003559,beloved
6522,0.000553,0.007117,textoperator


In [82]:
tokens['spam_ratio'] = tokens.spam / tokens.ham
tokens.sample(5, random_state=6)

,ham,spam,token,spam_ratio
6977,0.018247,0.007117,very,0.390057
4521,0.000829,0.005338,nasty,6.435943
6999,0.000553,0.005338,villa,9.653915
1266,0.000829,0.003559,beloved,4.290629
6522,0.000553,0.007117,textoperator,12.871886


In [83]:
tokens.sort_values('spam_ratio', ascending=False)

,ham,spam,token,spam_ratio
1766,0.000553,0.160142,claim,289.617438
5209,0.000553,0.137011,prize,247.783808
293,0.000553,0.088968,150p,160.898577
7357,0.000829,0.131673,www,158.753262
6680,0.000553,0.087189,tone,157.680605
3139,0.000553,0.078292,guaranteed,141.590747
307,0.000553,0.071174,18,128.718861
6829,0.000829,0.101423,uk,122.282918
2022,0.000553,0.067616,cs,122.282918
246,0.000553,0.058719,1000,106.193060


In [85]:
# doesn't work
#tokens.loc['dating', 'spam_ratio']

Part 8: Creating a DataFrame from individual text files

In [90]:
import glob
ham_filenames = glob.glob('../MLtext2/MLtext2/data/ham_files/*.txt')
ham_filenames

['../MLtext2/MLtext2/data/ham_files/email1.txt',
 '../MLtext2/MLtext2/data/ham_files/email3.txt',
 '../MLtext2/MLtext2/data/ham_files/email5.txt']

In [91]:
ham_text = []
for filename in ham_filenames:
    with open(filename) as f:
        ham_text.append(f.read())
ham_text

['This is a ham email.\nIt has 2 lines.\n',
 'This is another ham email.\n',
 'This is yet another ham email.\n']

In [92]:
spam_filenames = glob.glob('../MLtext2/MLtext2/data/spam_files/*.txt')
spam_filenames

['../MLtext2/MLtext2/data/spam_files/email2.txt',
 '../MLtext2/MLtext2/data/spam_files/email4.txt']

In [93]:
spam_text = []
for filename in spam_filenames:
    with open(filename) as f:
        spam_text.append(f.read())
spam_text

['This is a spam email.\n', 'This is another spam email.\n']

In [94]:
all_text = ham_text + spam_text
all_text

['This is a ham email.\nIt has 2 lines.\n',
 'This is another ham email.\n',
 'This is yet another ham email.\n',
 'This is a spam email.\n',
 'This is another spam email.\n']

In [95]:
all_labels = [0]*len(ham_text) + [1]*len(spam_text)
all_labels

[0, 0, 0, 1, 1]

In [96]:
pd.DataFrame({'label':all_labels, 'message':all_text})

,label,message
0,0,This is a ham email.\nIt has 2 lines.\n
1,0,This is another ham email.\n
2,0,This is yet another ham email.\n
3,1,This is a spam email.\n
4,1,This is another spam email.\n
